In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-mails-dataset/spam_ham_dataset.csv


In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import FunctionTransformer,LabelEncoder


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_csv("/kaggle/input/spam-mails-dataset/spam_ham_dataset.csv",index_col = 0)
df.drop('label_num' , axis = 1 ,inplace = True)
# df.columns = ['text' , 'target']
label_encoder = LabelEncoder()
df['target']= label_encoder.fit_transform(df['label'])
df.sample(5)

,label,text,target
733,ham,Subject: ( no subject )\r\nhey daren its me .\...,0
1560,ham,"Subject: enron / hpl actuals for sept . 5 , 20...",0
2140,ham,Subject: tri - c / cokinos gov bill daniels de...,0
1877,ham,Subject: cornhusker - cleburne plant outage\r\...,0
3710,spam,Subject: antigen downstairs dance\r\nstill no ...,1


In [4]:

def text_transform(desc):
    text = desc.lower()
    text = nltk.word_tokenize(text)
    words = [word for word in text if word.isalnum()]
    words = [word for word in words if word not in stopwords.words('english')]

    # Apply stemming using PorterStemmer
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]

    return " ".join(stemmed_words)

def data_preprocessing(df):
    df["transformed_text"] = df['text'].apply(text_transform)
    return df["transformed_text"]

In [5]:
X = df.drop(['target' , 'label'] ,axis = 1).copy()
y = df['target'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=2)

In [6]:
X.sample(5)

,text
586,Subject: hpl liquids / exxon\r\ndeal numbers :...
5020,Subject: meet some lonely fun girls that want ...
3053,Subject: re : noms / actual flow for 4 / 11 / ...
3994,Subject: better s . e . x guar . antee . d\r\n...
3108,"Subject: pleo 0004 2000\r\nmary ,\r\nas in my ..."


In [7]:
# Create a pipeline
pipeline = Pipeline([
    ('data_preprocessing', FunctionTransformer(func=data_preprocessing, validate=False)),
    ('vectorization', TfidfVectorizer(max_features=3000)),
    ('SVC', SVC(kernel='sigmoid', gamma=1.0))
])

In [8]:
# Train the pipeline
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.98


In [9]:
import pickle
with open('trained_pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

print("Trained pipeline saved to 'trained_pipeline.pkl'")

Trained pipeline saved to 'trained_pipeline.pkl'


In [10]:
test_data = pd.DataFrame(data = [X_test['text'].iloc[0]] , columns = ['text'])
test_data

,text
0,"Subject: enron actuals for july 14 thru 16 , 2..."


In [11]:
pipeline.predict(test_data)

array([0])